# Working with arrays
<!--
  ~ Licensed to the Apache Software Foundation (ASF) under one
  ~ or more contributor license agreements.  See the NOTICE file
  ~ distributed with this work for additional information
  ~ regarding copyright ownership.  The ASF licenses this file
  ~ to you under the Apache License, Version 2.0 (the
  ~ "License"); you may not use this file except in compliance
  ~ with the License.  You may obtain a copy of the License at
  ~
  ~   http://www.apache.org/licenses/LICENSE-2.0
  ~
  ~ Unless required by applicable law or agreed to in writing,
  ~ software distributed under the License is distributed on an
  ~ "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
  ~ KIND, either express or implied.  See the License for the
  ~ specific language governing permissions and limitations
  ~ under the License.
  -->

This notebook walks through functions and aggregation technique when using [arrays](https://druid.apache.org/docs/27.0.0/querying/sql-data-types/#arrays).

## Prerequisites

This tutorial works with Druid 27.0.0 or later.

#### Run with Docker

Launch this tutorial and all prerequisites using the `druid-jupyter` profile of the Docker Compose file for Jupyter-based Druid tutorials. For more information, see [Docker for Jupyter Notebook tutorials](https://druid.apache.org/docs/latest/tutorials/tutorial-jupyter-docker.html).
   

## Initialization

The following cells set up the notebook and learning environment ready for use.

### Set up and connect to the learning environment

Run the next cell to set up the Druid Python client's connection to Apache Druid.

If successful, the Druid version number will be shown in the output.

In [ ]:
import druidapi
import os

if 'DRUID_HOST' not in os.environ.keys():
    druid_host=f"http://localhost:8888"
else:
    druid_host=f"http://{os.environ['DRUID_HOST']}:8888"
    
print(f"Opening a connection to {druid_host}.")
druid = druidapi.jupyter_client(druid_host)

display = druid.display
sql_client = druid.sql
status_client = druid.status

status_client.version

### Load example data

Run the following cell to create a table called `example-wikipedia-arrays`. Notice {the use of X as a timestamp | only required columns are ingested | WHERE / expressions / GROUP BY are front-loaded | partitions on X period and clusters by Y}.

When completed, you'll see a description of the final table.

In [ ]:
sql='''
REPLACE INTO "example-wikipedia-arrays" OVERWRITE ALL
WITH "ext" AS (SELECT *
FROM TABLE(
  EXTERN(
    '{"type":"http","uris":["https://druid.apache.org/data/wikipedia.json.gz"]}',
    '{"type":"json"}'
  )
) EXTEND ("isRobot" VARCHAR, "channel" VARCHAR, "timestamp" VARCHAR, "flags" VARCHAR, "isUnpatrolled" VARCHAR, "page" VARCHAR, "diffUrl" VARCHAR, "added" BIGINT, "comment" VARCHAR, "commentLength" BIGINT, "isNew" VARCHAR, "isMinor" VARCHAR, "delta" BIGINT, "isAnonymous" VARCHAR, "user" VARCHAR, "deltaBucket" BIGINT, "deleted" BIGINT, "namespace" VARCHAR, "cityName" VARCHAR, "countryName" VARCHAR, "regionIsoCode" VARCHAR, "metroCode" BIGINT, "countryIsoCode" VARCHAR, "regionName" VARCHAR))
SELECT
  TIME_PARSE("timestamp") AS "__time",
  "channel",
  "commentLength"
FROM "ext"
PARTITIONED BY DAY
'''

display.run_task(sql)
sql_client.wait_until_ready('example-wikipedia-arrays')
display.table('example-wikipedia-arrays')

<!-- Include these cells if you need additional Python modules -->

### Import additional modules

Run the following cell to import additional Python modules that you will use to X, Y, Z.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

## Aggregating data to create arrays

Apache Druid includes several [aggregation functions](https://druid.apache.org/docs/27.0.0/querying/sql-aggregations/) that can be used as part of a GROUP BY to generate arrays.

In this first part of the notebook, you'll see some SQL queries on the sample data that you've loaded that generate arrays, and start to see some [array functions](https://druid.apache.org/docs/27.0.0/querying/sql-array-functions) being used.

### Create an array

The ARRAY_AGG aggregation function creates an array when doing a GROUP BY.

Run the following query, which brings the channels in the source table together into an array, one per 10 seconds, over a minute's worth of the data.

Notice that a second parameter to the ARRAY_AGG function is a maximum size for the array, specified here as 64KB.

In [ ]:
sql='''
SELECT
  TIME_FLOOR("__time",'PT10S') AS "timebucket",
  ARRAY_AGG("channel", 65535) AS "channel-array",
  SUM("commentLength") AS "total-commentLength"
FROM "example-wikipedia-arrays"
WHERE TIME_IN_INTERVAL("__time",'2016-06-27T04/PT1M')
GROUP BY 1
'''

display.sql(sql)

When you want to only maintain unique values in the array, include the DISTINCT keyword.

Run the query below to see what difference this makes to the results.

In [ ]:
sql='''
SELECT
  TIME_FLOOR("__time",'PT10S') AS "timebucket",
  ARRAY_AGG(DISTINCT "channel", 65535) AS "channel-array",
  SUM("commentLength") AS "total-commentLength"
FROM "example-wikipedia-arrays"
WHERE TIME_IN_INTERVAL("__time",'2016-06-27T04/PT1M')
GROUP BY 1
'''

display.sql(sql)

### Combine arrays

It's possible to concatenate arrays using the ARRAY_CONCAT_AGG aggregation function as part of a GROUP BY.

In the next cell you will see a query where the same results as above are then used to aggregate further - from 10 seconds to 10 minutes. The DISTINCT keyword is used in the ARRAY_CONCAT_AGG function to ensure only unique values are included in the array.

In [ ]:
sql='''
WITH "example-wikipedia-arrays-rollup" AS
( SELECT
  TIME_FLOOR("__time",'PT10S') AS "__time",
  ARRAY_AGG(DISTINCT "channel", 65535) AS "channel-array",
  SUM("commentLength") AS "commentLength"
FROM "example-wikipedia-arrays"
GROUP BY 1 )

SELECT
  TIME_FLOOR("__time",'PT10M') AS "timebucket",
  ARRAY_CONCAT_AGG(DISTINCT "channel-array",65535) AS "channel-array-combined-distinct",
  SUM("commentLength") AS "total-total-commentLength"
FROM "example-wikipedia-arrays-rollup"
WHERE TIME_IN_INTERVAL("__time",'2016-06-27T04/PT5M')
GROUP BY 1
'''

display.sql(sql)

## Using array functions

Now that we have a table containing arrays that we can refer to, the remainder of this notebook walks through various functions that are available.

### Find the number of elements in an array

ARRAY_LENGTH determines the number of elements in the array.

Take a look at the following SQL.

- The WITH statement uses a SELECT with GROUP BY to create arrays from the raw data you have ingested.
- The SELECT statement contains the ARRAY_LENGTH function to measure how long each array is.
- Each array provided to ARRAY_LENGTH is a concatenation of the arrays in the WITH statement.

The statement has been crafted this way to mimic what might happen if your source data itself contains arrays. The source data is represented by the "example-wikpedia-arrays-rollup" table generated by the WITH statement.

In [ ]:
sql='''
WITH "example-wikipedia-arrays-rollup" AS
( SELECT
  TIME_FLOOR("__time",'PT10S') AS "__time",
  ARRAY_AGG(DISTINCT "channel", 65535) AS "channel-array",
  SUM("commentLength") AS "commentLength"
FROM "example-wikipedia-arrays"
GROUP BY 1 )

SELECT
  TIME_FLOOR("__time",'PT15M') AS "timebucket",
  ARRAY_LENGTH(ARRAY_CONCAT_AGG(DISTINCT "channel-array",65535)) AS "channel-array-distinct-length",
  ARRAY_LENGTH(ARRAY_CONCAT_AGG("channel-array",65535)) AS "channel-array-length"
FROM "example-wikipedia-arrays-rollup"
WHERE TIME_IN_INTERVAL("__time",'2016-06-27T04/PT1H')
GROUP BY 1
'''

display.sql(sql)

### Test array contents

We can test whether a value exists inside an array by using ARRAY_CONTAINS.

Run the next cell to find the sum total comment length, calculated by adding together data from the "example-wikipedia-arrays-rollup" data only where the "channel-array" contains the "#de.wikipedia".

In [ ]:
sql='''
WITH "example-wikipedia-arrays-rollup" AS
( SELECT
  TIME_FLOOR("__time",'PT10S') AS "__time",
  ARRAY_AGG(DISTINCT "channel", 65535) AS "channel-array",
  SUM("commentLength") AS "commentLength"
FROM "example-wikipedia-arrays"
GROUP BY 1 )

SELECT
  SUM("commentLength") as "totalCommentLength"
FROM "example-wikipedia-arrays-rollup"
WHERE TIME_IN_INTERVAL("__time",'2016-06-27T04/PT1H')
AND ARRAY_CONTAINS("channel-array",'#de.wikipedia')
'''

display.sql(sql)

Use ARRAY_CONTAINS like other filters in combination with aggregate functions to calculate multiple filtered aggregates.

In [ ]:
sql='''
WITH "example-wikipedia-arrays-rollup" AS
( SELECT
  TIME_FLOOR("__time",'PT10S') AS "__time",
  ARRAY_AGG(DISTINCT "channel", 65535) AS "channel-array",
  SUM("commentLength") AS "commentLength"
FROM "example-wikipedia-arrays"
GROUP BY 1 )

SELECT
  SUM("commentLength") FILTER (WHERE ARRAY_CONTAINS("channel-array",'#de.wikipedia')) as "totalCommentLength-de",
  SUM("commentLength") FILTER (WHERE ARRAY_CONTAINS("channel-array",'#en.wikipedia')) as "totalCommentLength-en"
FROM "example-wikipedia-arrays-rollup"
WHERE TIME_IN_INTERVAL("__time",'2016-06-27T04/PT1H')
'''

display.sql(sql)

In [ ]:
### Find elements in an array

ARRAY_OFFSET
ARRAY_ORDINAL
ARRAY_SLICE

In [ ]:
### Add elements to an array

ARRAY_APPEND
ARRAY_PREPEND
ARRAY_CONCAT

## Clean up

Run the following cell to remove the XXX used in this notebook from the database.

In [ ]:
druid.datasources.drop("example-wikipedia-arrays")

## Summary

* You learned this
* Remember this

## Learn more

* Adapt the summarised table to use [Apache Datasketches](./approxCountDistinct.ipynb) for the user array
* Solve for problem X that is't covered here
* Read docs pages
* Watch or read something cool from the community
* Do some exploratory stuff on your own

In [ ]:
# STANDARD CODE BLOCKS

# When just wanting to display some SQL results
display.sql(sql)

# When ingesting data:
display.run_task(sql)
sql_client.wait_until_ready('example-wikipedia-arrays')
display.table('example-wikipedia-arrays')

# When you want to make an EXPLAIN look pretty
print(json.dumps(json.loads(sql_client.explain_sql(sql)['PLAN']), indent=2))

# When you want a simple plot
df = pd.DataFrame(sql_client.sql(sql))
df.plot(x='x-axis', y='y-axis', marker='o')
plt.xticks(rotation=45, ha='right')
plt.gca().get_legend().remove()
plt.show()

# When you want to add some query context parameters
req = sql_client.sql_request(sql)
req.add_context("useApproximateTopN", "false")
resp = sql_client.sql_query(req)

# When you want to compare two different sets of results
df3 = df1.compare(df2, keep_equal=True)
df3